In [ ]:
!pip3 install ultralytics easyocr rapidfuzz supervision levenshtein

In [ ]:
!pip3 uninstall opencv-python -y
!pip3 uninstall opencv-contrib-python -y
!pip3 install opencv-contrib-python

In [10]:
import cv2
import numpy as np
from ultralytics import YOLO
from PIL import Image
import easyocr
from rapidfuzz.distance import JaroWinkler
import supervision as sv
from tqdm import tqdm
from Levenshtein import distance as levenshtein_distance
from paddleocr import PaddleOCR


In [12]:
upscale_model = cv2.dnn_superres.DnnSuperResImpl_create()
upscale_model.readModel('/Users/shravanp/Coding/Robotics/AIScouter/src/models/ESPCN_x4.pb')
upscale_model.setModel('espcn', 4)

/notebooks/main


In [ ]:
model = YOLO('/Users/shravanp/Coding/Robotics/AIScouter/src/models/y8v7.pt')

# Initialize EasyOCR reader
# reader = easyocr.Reader(['en'], gpu=True)
ocr = PaddleOCR(use_angle_cls=True, lang='en')
# Define the team numbers for red and blue teams
original_red_team_numbers = ["75", "2722", "1391"]  # Replace with actual red team numbers
original_blue_team_numbers = ["56", "5401", "8513"]  # Replace with actual blue team numbers

In [ ]:
video_path = "/Users/shravanp/Coding/Robotics/AIScouter/src/videos/dcmp58-3sec.mp4"
output_path = "/Users/shravanp/Coding/Robotics/AIScouter/src/videos/dcmp58-3sec-test.mp4"
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

bounding_box_annotator = sv.BoxAnnotator()

frame_count = 0
        
with tqdm(total=int(cap.get(cv2.CAP_PROP_FRAME_COUNT))) as pbar:
    while True:
        ret, frame = cap.read()
        red_team_numbers = original_red_team_numbers.copy()
        blue_team_numbers = original_blue_team_numbers.copy()
        
        if not ret:
            print("Finished processing all frames.")
            break

        if frame is None:
            print("Empty frame encountered.")
            continue

        # Convert the frame from BGR (OpenCV) to RGB (PIL)
        image_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        results = model(image_pil)

        blue_boxes = []
        red_boxes = []

        # Separate boxes by team color
        for result in results:
            boxes = result.boxes
            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                start_row = (y1 + y2) // 2
                bottom_half = image[start_row:y2, x1:x2]

                # Calculate average RGB to determine the team color
                average_rgb = np.mean(bottom_half, axis=(0, 1))
                label = 'Blue' if average_rgb[0] > average_rgb[2] else 'Red'
                color = (255, 0, 0) if label == 'Blue' else (0, 0, 255)

                # OCR: Recognize text in the bottom half of the box
                upscaled_image = upscale_model.upsample(bottom_half)
                gray = cv2.cvtColor(upscaled_image, cv2.COLOR_BGR2GRAY)
                clahe = cv2.createCLAHE(clipLimit=2, tileGridSize=(30,30))
                enhanced_gray = clahe.apply(gray)

                result = ocr.ocr(np.array(enhanced_gray), cls=False)
                ocr_result = ""
                try:
                    ocr_result = result[0][0][1][0]
                except:
                    ocr_result = ""

                detected_text = ocr_result.replace(" ", "") if ocr_result else ""

                if label == 'Blue':
                    blue_boxes.append((x1, y1, x2, y2, detected_text, color))
                else:
                    red_boxes.append((x1, y1, x2, y2, detected_text, color))

        def assign_team_numbers(boxes, team_numbers, previous_assignments=None):
            # Create a copy of the team numbers to avoid modifying the original list
            available_team_numbers = team_numbers[:]
            assigned_numbers = set()  # Track used numbers in this frame

            # Create a mapping of detected boxes to team numbers
            box_to_team_mapping = {}

            for x1, y1, x2, y2, detected_text, color in boxes:
                if detected_text:
                    # Calculate Levenshtein distances
                    distances = {num: levenshtein_distance(detected_text, num) for num in available_team_numbers}
                    # Sort distances by closest match
                    sorted_distances = sorted(distances.items(), key=lambda item: item[1])

                    detected_team_number = None
                    for team_number, _ in sorted_distances:
                        if team_number not in assigned_numbers:
                            detected_team_number = team_number
                            assigned_numbers.add(team_number)
                            available_team_numbers.remove(team_number)
                            break

                    # If no match found, fallback to the first available number
                    if detected_team_number is None and available_team_numbers:
                        detected_team_number = available_team_numbers.pop(0)
                        assigned_numbers.add(detected_team_number)
                else:
                    # Handle cases with no detected text
                    if available_team_numbers:
                        detected_team_number = available_team_numbers.pop(0)
                        assigned_numbers.add(detected_team_number)
                    else:
                        detected_team_number = "Unknown"  # Fallback if all numbers are used (should not happen with correct data)

                # Map the box to the detected team number
                box_to_team_mapping[(x1, y1, x2, y2)] = detected_team_number

            # If you have previous frame assignments, you can interpolate to handle missing data
            if previous_assignments:
                for box in box_to_team_mapping:
                    if box_to_team_mapping[box] == "Unknown":
                        # Attempt to interpolate from previous frame's data
                        box_to_team_mapping[box] = previous_assignments.get(box, "Unknown")

            # Draw the bounding boxes and labels on the image
            for (x1, y1, x2, y2), detected_team_number in box_to_team_mapping.items():
                # Draw the bounding box and label on the image
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"{detected_team_number}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

            return box_to_team_mapping  # Return mapping for potential interpolation in next frame
        
        
        # Assign team numbers to Blue and Red boxes
        assign_team_numbers(blue_boxes, blue_team_numbers)
        assign_team_numbers(red_boxes, red_team_numbers)

        # Write the frame with annotations to the output video
        out.write(frame)
        frame_count += 1
        pbar.update(1)

        
# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video processing complete. Output saved to {output_path}")